In [25]:
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict
import datetime
import threading

In [26]:
src_root_dir = "month"
des_root_dir = "newmonth"

def processSingleDir(directory):
    aircraftDict = defaultdict(list)
    index = int(directory[8: 10])
    desDirName = "day" + str(index)
    currTargetDir = os.path.join(src_root_dir, directory)
    currDataList = os.listdir(currTargetDir)
    for currDataName in currDataList:
        currDataPath = os.path.join(currTargetDir, currDataName)
        with open(currDataPath) as currJson:
            currData = json.load(currJson)
        aircraftLen = len(currData["acList"])
        for idx in range(aircraftLen):
            currAircraft = currData["acList"][idx] #获取当前飞机数据
            currAircraft["timeRange"] = currDataName
            timestamp = int(currAircraft["FSeen"][6:-2])  #将FSeen信息格式化
            currAircraft["FSeen"] = datetime.datetime.fromtimestamp(timestamp/1000).strftime('%H:%M:%S')

            timestamp = int(currAircraft["PosTime"])   #将PosTime信息格式化
            currAircraft["PosTime"] = datetime.datetime.fromtimestamp(timestamp/1000).strftime('%H:%M:%S')
            
            currId = currAircraft["Id"]
            aircraftDict[currId].append(currAircraft)
            
    keys = list(aircraftDict.keys())
    for key in keys:
        aircraftDict[key] = pd.DataFrame(aircraftDict[key])
        aircraftDict[key] = aircraftDict[key].sort_values("FSeen", ascending=True)
        currColumns = list(aircraftDict[key])
        currColumns.remove("timeRange"); currColumns.remove("FSeen"); currColumns.remove("PosTime")
        currColumns.insert(1, "PosTime"); currColumns.insert(1, "FSeen"); currColumns.insert(1, "timeRange")
        aircraftDict[key] = aircraftDict[key].reindex(columns=currColumns)
        aircraftDict[key].to_csv("{}//day{}//aircraft{}.csv".format(des_root_dir,index, key), index=False)
        del aircraftDict[key]
    

In [27]:
threads = []

for idx in range(0, 30, 7):
    
    end = idx + 7
    if idx + 7 > 30:
        end = 30
    for directory in os.listdir(src_root_dir)[idx:idx+end]:
        thread = threading.Thread(target=processSingleDir, args=(directory,))
        thread.start()
        threads.append(thread)
        
    for thread in threads:
        thread.join()
    threads.clear()
print("结束")


All threads have finished processing.
